In [32]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from time import time
from numba import jit

@jit
def LJ(r):
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

@jit
def energy(points): #points is a 3 * N array of all the atoms involved
    total = 0
    num_atoms = int(len(points)/3)
    
    for i in range(num_atoms - 1):
        for j in range(i+1, num_atoms):
            pos1 = points[i*3 : (i+1)*3]
            pos2 = points[j*3 : (j+1)*3]
            
            r = np.linalg.norm(pos1-pos2)
            
            total += LJ(r)
    return total

@jit #For some reason I get a warning whenever I use this on this function, seems like it's to do with np.zeros?
def gradient(pos):
    N = int(len(pos)/3)
    J = np.zeros( [N, 3])
    
    for i in range(N-1):
        center = pos[3*i: (3*i)+3 ]
        for j in range(i+1, N):
            neighbor = pos[3*j: (3*j)+3 ]
            r = np.linalg.norm(center - neighbor)
            J[i] += (24/r**7 - 48/r**13) * (center - neighbor)/r
            
    return J.flatten()

def make_points(num_atoms):
    return 5*np.random.random_sample((num_atoms*3,))#For some reason this needs to be multiplied by a large enough constant

atoms = 12
points = make_points(atoms)

min_energy = -37.9 + 2 #Whatever it is, according to the document we were given. The second term is the tolerance

trials = 20
avg = 0
E = []
i = 0
for i in range(trials):
    t0 = time()
    points = make_points(atoms)
    res = minimize(energy, points, method='Powell', tol=1e-4, options={"maxiter":20})
    res = minimize(energy, res.x, method='CG', jac=gradient, tol=1e-4) #Other methods include BFGS or COBYLA
    E.append(res.fun)
    print("Trial: " + str(i+1) + "  Energy: " + str(res.fun) + "  Time: " + str(time()-t0) )

#print(avg/trials)
print(min(E))

C:\Users\samse\AppData\Local\Temp/ipykernel_6744/1253441630.py:28: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "gradient" failed type inference due to: No implementation of function Function(<built-in function zeros>) found for signature:
 
 >>> zeros(list(int64)<iv=None>)
 
There are 4 candidate implementations:
      - Of which 4 did not match due to:
      Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: numba\core\overload_glue.py: Line 131.
        With argument(s): '(list(int64)<iv=None>)':
       Rejected as the implementation raised a specific error:
         TypingError: Failed in nopython mode pipeline (step: nopython frontend)
       No implementation of function Function(<intrinsic stub>) found for signature:
        
        >>> stub(list(int64)<iv=None>)
        
       There are 2 candidate implementations:
         - Of which 2 did not match due to:
         Intrinsic of function 'stub': 

Trial: 1  Energy: -24.55865719792275  Time: 4.5875232219696045
Trial: 2  Energy: -24.24777196239376  Time: 0.8299305438995361
Trial: 3  Energy: -17.6711609415406  Time: 0.8503282070159912
Trial: 4  Energy: -33.84938853865876  Time: 0.8252673149108887
Trial: 5  Energy: -29.433382500156043  Time: 0.8259916305541992
Trial: 6  Energy: -28.305852930712092  Time: 0.8389594554901123
Trial: 7  Energy: -32.50254126187267  Time: 0.822547435760498
Trial: 8  Energy: -23.08552451580281  Time: 0.8275461196899414
Trial: 9  Energy: -24.18333586516003  Time: 0.8302514553070068
Trial: 10  Energy: -32.80427054721351  Time: 0.8412899971008301
Trial: 11  Energy: -19.5802066326384  Time: 0.7983884811401367
Trial: 12  Energy: -22.8097171766917  Time: 0.8860208988189697
Trial: 13  Energy: -24.49856667192083  Time: 0.8289525508880615
Trial: 14  Energy: -17.777848720532596  Time: 0.8592455387115479
Trial: 15  Energy: -33.303122547444445  Time: 0.8153131008148193
Trial: 16  Energy: -23.14104566174999  Time: 0.84

No matter how I seem to code the average number of attempts to find the ground state, it keeps on getting stuck on the same local minimum over and over, even when I rearrange the points. I'm sure I would find more variance with a higher number of atoms, but at least when numbers are relatively small it seems to either get it or not get it all. However, as the numbers get higher, the variance does increase enough that the optimization methods achieve different minimums, allowing the average to be more than one.

The relation we see for the Jacobian is gotten by taking the partial derivative of the LJ equation with respect to r and then multiplying it by the unit vector of the change in position from the current position to the neighbor. That way, the difference in position is accounted for.